In [1]:
# get our db url
# create db engine via sqlalchemy

In [2]:
from env import config, BASE_DIR
from sqlalchemy import create_engine, text

In [3]:
DATABASE_URL = config("HOUSING_DATABSE_URL", default=None)
DATABASE_URL is not None

True

In [4]:
engine = create_engine(str(DATABASE_URL))

In [5]:
DATABASE_DIR = BASE_DIR / "datasets"
DATABASE_DIR.exists()

True

In [6]:
HOUSES_CSV = DATABASE_DIR / "housing_ds.csv"
HOUSES_CSV.exists()

True

In [7]:
import pandas as pd # dask for larger datasets
# from dask import dataframe as dd

In [8]:
df = pd.read_csv(str(HOUSES_CSV))
df.head(n=10)

,saledate,MA,type,bedrooms
0,30/09/2007,441854,house,2
1,31/12/2007,441854,house,2
2,31/03/2008,441854,house,2
3,30/06/2008,441854,house,2
4,30/09/2008,451583,house,2
5,31/12/2008,440256,house,2
6,31/03/2009,442566,house,2
7,30/06/2009,446113,house,2
8,30/09/2009,440123,house,2
9,31/12/2009,442131,house,2


In [9]:
rows, cols = df.shape
rows, cols

(347, 4)

In [12]:
df.dtypes

saledate    object
MA           int64
type        object
bedrooms     int64
dtype: object

In [15]:
df['date'] = pd.to_datetime(df['saledate'], format='%d/%m/%Y')

In [16]:
df.dtypes

saledate            object
MA                   int64
type                object
bedrooms             int64
date        datetime64[ns]
dtype: object

In [18]:
df['price'] = df['MA']

In [19]:
out_df = df.copy()[['date', 'price', 'type', 'bedrooms']]

In [21]:
df.head(n=1)

,saledate,MA,type,bedrooms,date,price
0,30/09/2007,441854,house,2,2007-09-30,441854


In [22]:
out_df.head(n=1)

,date,price,type,bedrooms
0,2007-09-30,441854,house,2


In [23]:
table_name = "kaggle_house_data"
out_df.to_sql(table_name, engine, if_exists="replace", index=False)

347

In [24]:
# table_name = "kaggle_house_data"
# df.to_sql(table_name, engine, if_exists="replace", index=False)

In [25]:
with engine.connect() as connection:
    result = connection.execute(text(f"SELECT * FROM {table_name} LIMIT 10"))
    print(result.all())

[(datetime.datetime(2007, 9, 30, 0, 0), 441854, 'house', 2), (datetime.datetime(2007, 12, 31, 0, 0), 441854, 'house', 2), (datetime.datetime(2008, 3, 31, 0, 0), 441854, 'house', 2), (datetime.datetime(2008, 6, 30, 0, 0), 441854, 'house', 2), (datetime.datetime(2008, 9, 30, 0, 0), 451583, 'house', 2), (datetime.datetime(2008, 12, 31, 0, 0), 440256, 'house', 2), (datetime.datetime(2009, 3, 31, 0, 0), 442566, 'house', 2), (datetime.datetime(2009, 6, 30, 0, 0), 446113, 'house', 2), (datetime.datetime(2009, 9, 30, 0, 0), 440123, 'house', 2), (datetime.datetime(2009, 12, 31, 0, 0), 442131, 'house', 2)]


(347, 4)